In [42]:
import pandas as pd
import numpy as np
import json
import copy
from datetime import datetime, timedelta

from lolchess.model.match import GameType, Match
from lolchess.model.participant import Participant
from lolchess.model.analyze_set5 import AnalyzeSet5
from lolchess.DatabaseManager import DatabaseManager, PostgresDatabaseManager
from lolchess.StaticDataManager import StaticDataManager

from sqlalchemy import exists, and_, or_

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from Configuration import *

In [7]:
from sklearn.cluster import MeanShift

In [85]:
%reload_ext autoreload
%autoreload 2

In [3]:
class Analyzer:
    def __init__(self):
        self.tft_clustering_version = 1
        self.db_manager = DatabaseManager()
        self.postgres_db_manager = PostgresDatabaseManager()
        self.staticdata_manager = StaticDataManager()
        self.daily_timedelta = timedelta(days=1)
        self.period_timedelta = timedelta(hours=6)
        
        self.df_match_info = None
        self.df_read = None
        self.df_cluster = None

    '''
    duration_datetime:
        
    params:
        
    returns:
        
    '''
    def duration_datetime(self, start_time, end_time, period_delta):
        previous_time = start_time
        current_time = start_time + period_delta
        while not current_time > end_time:
            yield previous_time, current_time
            previous_time = current_time
            current_time = current_time + period_delta
            
    '''
    '''
    def read_data(self, start_time, end_time, limit=None):
        # ----------------------------------------------------
        #
        # Read match information by participants
        #
        # ----------------------------------------------------
        if limit == None:
            self.df_match_info = pd.read_sql(self.db_manager.session.query(Match) \
                    .filter(Match.setnumber == 5) \
                    .filter(and_(Match.matched_at <= end_time, Match.matched_at >= start_time)).statement, \
                self.db_manager.session.bind)
            self.df_read = pd.read_sql(self.db_manager.session.query(AnalyzeSet5, Participant) \
                    .join(Participant) \
                    .join(Match) \
                    .filter(Participant.is_analyzed == True) \
                    .filter(Match.setnumber == 5) \
                    .filter(and_(Match.matched_at <= end_time, Match.matched_at >= start_time)).statement, \
                self.db_manager.session.bind)
        else:
            self.df_match_info = pd.read_sql(self.db_manager.session.query(Match) \
                    .filter(Match.setnumber == 5) \
                    .filter(and_(Match.matched_at <= end_time, Match.matched_at >= start_time)).limit(limit).statement, \
                self.db_manager.session.bind)
            self.df_read = pd.read_sql(self.db_manager.session.query(AnalyzeSet5, Participant) \
                    .join(Participant) \
                    .join(Match) \
                    .filter(Participant.is_analyzed == True) \
                    .filter(Match.setnumber == 5) \
                    .filter(and_(Match.matched_at <= end_time, Match.matched_at >= start_time)).limit(limit).statement, \
                self.db_manager.session.bind)
        self.df_clustering = self.df_read.drop(['id', 
                                  'last_round', 
                                  'level', 
                                  'placement', 
                                  'players_eliminated', 
                                  'time_eliminated', 
                                  'total_damage_to_players', 
                                  'traits', 
                                  'champions', 
                                  'is_analyzed', 
                                  'match_id', 
                                  'participant_id', 
                                  'id_1', 
                                  'gold_left'], axis=1)
        return self.df_clustering
    
    '''
    '''
    def set_centroids(self, labels, n_labels, ):
    
            # ----------------------------------------------------
            #
            # Caculate centroid and distance from centroid
            #
            # ----------------------------------------------------
            mean_result = df_dbscan_result[df_dbscan_result.label == deck_label].mean()
            df_result.loc[index_label, 'centroid_x'] = mean_result[0]
            df_result.loc[index_label, 'centroid_y'] = mean_result[1]
            df_result.loc[index_label, 'centroid_z'] = mean_result[2]

            df_cur_label = df_result[df_result.label == deck_label]
            df_cur_dbscan = df_dbscan_result[df_dbscan_result.label == deck_label]
            min_distance_id = -1
            min_distance_value = 1000
            
            df_cur_dbscan['distance'] = np.sum((np.array(df_cur_label[['centroid_x', '']])))

            for idx, row in df_dbscan_result[df_dbscan_result.label == deck_label].iterrows():
                if np.sum((np.array(df_cur_label[['centroid_x', 'centroid_y', 'centroid_z']]) - np.array(row[[0,1,2]])), axis=1) ** 2 < min_distance_value:
                    min_distance_value = np.sum((np.array(df_cur_label[['centroid_x', 'centroid_y', 'centroid_z']]) - np.array(row[[0,1,2]])), axis=1) ** 2
                    min_distance_id = idx
            if min_distance_id > -1:
                json_champions = json.loads(df_read.loc[min_distance_id, "champions"].replace("'", '"'))
                json_traits = json.loads(df_read.loc[min_distance_id, "traits"].replace("'", '"'))
                names = [self.staticdata_manager.GetChampionKoreanName(champion['character_id']) for champion in json_champions]
                df_result.loc[index_label, 'champions'] = str(names)
                names.clear()
                for trait in json_traits:
                    if trait['tier_current'] > 0:
                        names.append(self.staticdata_manager.GetTraitKoreanName(trait['name']))
                df_result.loc[index_label, 'traits'] = str(names)
                
    '''
    '''
    def run(self):
        pass
    
    '''
    '''
    def estimate_win_rate(self, start_time, end_time, count_cluster_labels):
        df_result = pd.DataFrame(count_cluster_labels)
        df_matchtime_gametype = pd.merge(df_read[['placement','match_id', 'label']], df_match_info[['id','matched_at', 'gametype_id']], how='left', left_on='match_id', right_on='id')

        for deck in count_cluster_labels:
            deck_label = deck['label']
            index_label = deck['label'] + 1

            # ----------------------------------------------------
            #
            # win_reate, defense_rate
            #
            # ----------------------------------------------------
            if len(df_read[df_read.label == deck_label].index) == 0:
                df_result.loc[index_label, 'win_rate'] = 0.00
                df_result.loc[index_label, 'defence_rate'] = 0.00
            else:
                df_result.loc[index_label, 'win_rate'] = round(len(df_read[(df_read.label == deck_label) & 
                                                                              (df_read.placement == 1)].index) / 
                                                                  len(df_read[df_read.label == deck_label].index) * 100, 2)
                df_result.loc[index_label, 'defence_rate'] = round(len(df_read[(df_read.label == deck_label) & 
                                                                                  (df_read.placement <= 4)].index) / 
                                                                      len(df_read[df_read.label == deck_label].index) * 100, 2)

            if len(df_matchtime_gametype[(df_matchtime_gametype.label == deck_label)&
                                        (df_matchtime_gametype.gametype_id == 1)].index) == 0:
                df_result.loc[index_label, 'turbo_win_rate'] = 0.00
                df_result.loc[index_label, 'turbo_defence_rate'] = 0.00
            else:
                df_result.loc[index_label, 'turbo_win_rate'] = round(len(df_matchtime_gametype[(df_matchtime_gametype.label == deck_label) &
                                                                                                (df_matchtime_gametype.placement == 1) &
                                                                                                (df_matchtime_gametype.gametype_id == 1)].index) /
                                                                        len(df_matchtime_gametype[(df_matchtime_gametype.label == deck_label) &
                                                                                                (df_matchtime_gametype.gametype_id == 1)].index) * 100, 2)
                df_result.loc[index_label, 'turbo_defence_rate'] = round(len(df_matchtime_gametype[(df_matchtime_gametype.label == deck_label) & 
                                                                                                    (df_matchtime_gametype.placement <= 4) &
                                                                                                    (df_matchtime_gametype.gametype_id == 1)].index) / 
                                                                            len(df_matchtime_gametype[(df_matchtime_gametype.label == deck_label) &
                                                                                                    (df_matchtime_gametype.gametype_id == 1)].index) * 100, 2)

            if len(df_matchtime_gametype[(df_matchtime_gametype.label == deck_label)&
                                        (df_matchtime_gametype.gametype_id == 2)].index) == 0:
                df_result.loc[index_label, 'standard_win_rate'] = 0.00
                df_result.loc[index_label, 'standard_defence_rate'] = 0.00
            else:
                df_result.loc[index_label, 'standard_win_rate'] = round(len(df_matchtime_gametype[(df_matchtime_gametype.label == deck_label) &
                                                                                                (df_matchtime_gametype.placement == 1) &
                                                                                                (df_matchtime_gametype.gametype_id == 2)].index) /
                                                                        len(df_matchtime_gametype[(df_matchtime_gametype.label == deck_label) &
                                                                                                (df_matchtime_gametype.gametype_id == 2)].index) * 100, 2)
                df_result.loc[index_label, 'standard_defence_rate'] = round(len(df_matchtime_gametype[(df_matchtime_gametype.label == deck_label) & 
                                                                                                    (df_matchtime_gametype.placement <= 4) &
                                                                                                    (df_matchtime_gametype.gametype_id == 2)].index) / 
                                                                            len(df_matchtime_gametype[(df_matchtime_gametype.label == deck_label) &
                                                                                                    (df_matchtime_gametype.gametype_id == 2)].index) * 100, 2)

            period_win_rate = []
            period_defence_rate = []
            turbo_period_win_rate = []
            turbo_period_defence_rate = []
            standard_period_win_rate = []
            standard_period_defence_rate = []
            daily_win_rate = []
            daily_defence_rate = []
            turbo_daily_win_rate = []
            turbo_daily_defence_rate = []
            standard_daily_win_rate = []
            standard_daily_defence_rate = []
            for s_time, e_time in self.duration_datetime(start_time, end_time, self.period_timedelta):
                df_matches = df_matchtime_gametype[(df_matchtime_gametype.label == deck_label) & 
                                            (df_matchtime_gametype.matched_at >= s_time) & 
                                            (df_matchtime_gametype.matched_at < e_time)]
                df_turbo_matches = df_matches[df_matches.gametype_id == 1]
                df_standard_matches = df_matches[df_matches.gametype_id == 2]

                if len(df_matches.index) == 0:
                    period_win_rate.append(0.00)
                    period_defence_rate.append(0.00)
                else:
                    period_win_rate.append(round(len(df_matches[(df_matches.placement == 1)].index) / len(df_matches.index) * 100, 2))
                    period_defence_rate.append(round(len(df_matches[(df_matches.placement <= 4)].index) / len(df_matches.index) * 100, 2))
                if len(df_turbo_matches.index) == 0:
                    turbo_period_win_rate.append(0.00)
                    turbo_period_defence_rate.append(0.00)
                else:
                    turbo_period_win_rate.append(round(len(df_turbo_matches[(df_turbo_matches.placement == 1)].index) / len(df_turbo_matches.index) * 100, 2))
                    turbo_period_defence_rate.append(round(len(df_turbo_matches[(df_turbo_matches.placement <= 4)].index) / len(df_turbo_matches.index) * 100, 2))
                if len(df_standard_matches.index) == 0:
                    standard_period_win_rate.append(0.00)
                    standard_period_defence_rate.append(0.00)
                else:
                    standard_period_win_rate.append(round(len(df_standard_matches[(df_standard_matches.placement == 1)].index) / len(df_standard_matches.index) * 100, 2))
                    standard_period_defence_rate.append(round(len(df_standard_matches[(df_standard_matches.placement <= 4)].index) / len(df_standard_matches.index) * 100, 2))

            for s_time, e_time in self.duration_datetime(start_time, end_time, self.daily_timedelta):
                df_matches = df_matchtime_gametype[(df_matchtime_gametype.label == deck_label) & 
                                            (df_matchtime_gametype.matched_at >= s_time) & 
                                            (df_matchtime_gametype.matched_at < e_time)]
                df_turbo_matches = df_matches[df_matches.gametype_id == 1]
                df_standard_matches = df_matches[df_matches.gametype_id == 2]

                if len(df_matches.index) == 0:
                    daily_win_rate.append(0.00)
                    daily_defence_rate.append(0.00)
                else:
                    daily_win_rate.append(round(len(df_matches[(df_matches.placement == 1)].index) /len(df_matches.index) * 100, 2))
                    daily_defence_rate.append(round(len(df_matches[(df_matches.placement <= 4)].index) /len(df_matches.index) * 100, 2))
                if len(df_turbo_matches.index) == 0:
                    turbo_daily_win_rate.append(0.00)
                    turbo_daily_defence_rate.append(0.00)
                else:
                    turbo_daily_win_rate.append(round(len(df_turbo_matches[(df_turbo_matches.placement == 1)].index) / len(df_turbo_matches.index) * 100, 2))
                    turbo_daily_defence_rate.append(round(len(df_turbo_matches[(df_turbo_matches.placement <= 4)].index) / len(df_turbo_matches.index) * 100, 2))
                if len(df_standard_matches.index) == 0:
                    standard_daily_win_rate.append(0.00)
                    standard_daily_defence_rate.append(0.00)
                else:
                    standard_daily_win_rate.append(round(len(df_standard_matches[(df_standard_matches.placement == 1)].index) / len(df_standard_matches.index) * 100, 2))
                    standard_daily_defence_rate.append(round(len(df_standard_matches[(df_standard_matches.placement <= 4)].index) / len(df_standard_matches.index) * 100, 2))

            df_result.loc[index_label, 'period_win_rate'] = str(period_win_rate)
            df_result.loc[index_label, 'period_defence_rate'] = str(period_defence_rate)
            df_result.loc[index_label, 'turbo_period_win_rate'] = str(turbo_period_win_rate)
            df_result.loc[index_label, 'turbo_period_defence_rate'] = str(turbo_period_defence_rate)
            df_result.loc[index_label, 'standard_period_win_rate'] = str(standard_period_win_rate)
            df_result.loc[index_label, 'standard_period_defence_rate'] = str(standard_period_defence_rate)
            df_result.loc[index_label, 'daily_win_rate'] = str(daily_win_rate)
            df_result.loc[index_label, 'daily_defence_rate'] = str(daily_defence_rate)
            df_result.loc[index_label, 'turbo_daily_win_rate'] = str(turbo_daily_win_rate)
            df_result.loc[index_label, 'turbo_daily_defence_rate'] = str(turbo_daily_defence_rate)
            df_result.loc[index_label, 'standard_daily_win_rate'] = str(standard_daily_win_rate)
            df_result.loc[index_label, 'standard_daily_defence_rate'] = str(standard_daily_defence_rate)

        result = [{
            'info' : {
                'version': self.tft_clustering_version,
                'start_time': start_time.strftime('%Y-%m-%d %H:%M:%S'),
                'end_time': end_time.strftime('%Y-%m-%d %H:%M:%S'),
                'period': i_period_win_rate,
                'num_labels': dbscan_n_clusters_
            },
            'data' : {
                'label': df_result['label'].to_list(),
                'counts': df_result['counts'].to_list(),
                'traits': df_result['traits'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'champions': df_result['champions'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'centroid_x': df_result['centroid_x'].to_list(),
                'centroid_y': df_result['centroid_y'].to_list(),
                'centroid_z': df_result['centroid_z'].to_list(),
                'win_rate': df_result['win_rate'].to_list(),
                'defence_rate': df_result['defence_rate'].to_list(),
                'period_win_rate': df_result['period_win_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'period_defence_rate': df_result['period_defence_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'turbo_period_win_rate': df_result['turbo_period_win_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'turbo_period_defence_rate': df_result['turbo_period_defence_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'standard_period_win_rate': df_result['standard_period_win_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'standard_period_defence_rate': df_result['standard_period_defence_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'daily_win_rate': df_result['daily_win_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'daily_defence_rate': df_result['daily_defence_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'turbo_daily_win_rate': df_result['turbo_daily_win_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'turbo_daily_defence_rate': df_result['turbo_daily_defence_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'standard_daily_win_rate': df_result['standard_daily_win_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
                'standard_daily_defence_rate': df_result['standard_daily_defence_rate'].apply(lambda x: json.loads(x.replace("'", '"'))).to_list(),
            }
        }]
        
        return result

In [64]:
analyzer = Analyzer()
analyze_start_date = datetime(2021, 5, 24, 0, 0, 0)
analyze_end_date = datetime(2021, 5, 31, 0, 0, 0)
df_cluster = analyzer.read_data(analyze_start_date, analyze_end_date)

# Cluster

In [65]:
# ----------------------------------------------------
#
# Normalizing Data
#
# ----------------------------------------------------
#df_c = StandardScaler().fit_transform(df_cluster)
df_c = MinMaxScaler().fit_transform(df_cluster)

# ----------------------------------------------------
#
# Clustering
#
# ----------------------------------------------------
#model_MeanShift = MeanShift(bandwidth=13).fit(df_c)
model_MeanShift = MeanShift(bandwidth=3.5).fit(df_c)
#db_model = DBSCAN(eps=7.142, min_samples=30).fit(df_c)

labels = model_MeanShift.labels_
len_labels = len(set(labels))
#colors = [[x/len_labels , x/len_labels, x/len_labels] for x in labels]

print('Estimated number of clusters: %d' % len_labels)
# ----------------------------------------------------
#
# Graph
#
# ----------------------------------------------------
X_db_fit_pca = PCA(n_components=3).fit_transform(df_c)
df_fit_pca = pd.DataFrame(X_db_fit_pca)

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df_fit_pca[0], df_fit_pca[1], df_fit_pca[2], 
            cmap='viridis', c=labels,
            edgecolor='k', s=40, alpha = 0.5)

ax.dist = 10

plt.autoscale(enable=True, axis='x', tight=True)    

plt.show()

KeyboardInterrupt: 

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_db_fit_pca[0],X_db_fit_pca[1],X_db_fit_pca[2], 
            c=labels, cmap='viridis',
            edgecolor='k', s=40, alpha = 0.5)

ax.dist = 10

plt.autoscale(enable=True, axis='x', tight=True)    

plt.show()

In [ ]:
count_db_labels = []
unique, counts = np.unique(db_labels, return_counts=True)
for u, c in zip(unique, counts):
    print('label ' + str(u) + ": " + str(c))
    count_db_labels.append(dict({'label': u, 'counts': c}))
    
df_db_result = pd.DataFrame(X_db_fit_pca)
df_db_result['label'] = pd.Series(db_labels, name='label')
df_read['label'] = pd.Series(db_labels, name='label')
df_db_result['id'] = df_read['id']
df_remove_noise = df_db_result.loc[df_db_result['label'] > -1]


In [8]:
start_time = datetime(2021, 5, 18, 0, 0, 0)
end_time = datetime(2021, 5, 19, 0, 0, 0)
period_win_rate = 6
tft_clustering_version = "00001"

period_timedelta = timedelta(hours=period_win_rate)
daily_timedelta = timedelta(days=1)
json_champions = None
json_traits = None
df_db_labels = pd.DataFrame(count_db_labels)
df_matchtime_gametype = pd.merge(df_read[['placement','match_id', 'label']], df_match_info[['id','matched_at', 'gametype_id']], how='left', left_on='match_id', right_on='id')

def duration_datetime(start_time, end_time, period_delta):
    previous_time = start_time
    current_time = start_time + period_delta
    while not current_time > end_time:
        yield previous_time, current_time
        previous_time = current_time
        current_time = current_time + period_delta

for dic in count_db_labels:
    dic_label = dic['label']
    index_label = dic['label'] + 1
    
    #df_db_labels['champions'] = np.nan
    #df_db_labels['champions'] = df_db_labels['champions'].astype(object)
    #df_db_labels['traits'] = np.nan
    #df_db_labels['traits'] = df_db_labels['traits'].astype(object)
    #df_db_labels['period_win_rate'] = np.nan
    #df_db_labels['period_win_rate'] = df_db_labels['period_win_rate'].astype(object)
    #df_db_labels['period_defence_rate'] = np.nan
    #df_db_labels['period_defence_rate'] = df_db_labels['period_defence_rate'].astype(object)
    #df_db_labels['daily_win_rate'] = np.nan
    #df_db_labels['daily_win_rate'] = df_db_labels['daily_win_rate'].astype(object)
    #df_db_labels['daily_defence_rate'] = np.nan
    #df_db_labels['daily_defence_rate'] = df_db_labels['daily_defence_rate'].astype(object)
    
    mean_result = df_db_result[df_db_result.label == dic_label].mean()
    df_db_labels.loc[index_label, 'centroid_x'] = mean_result[0]
    df_db_labels.loc[index_label, 'centroid_y'] = mean_result[1]
    df_db_labels.loc[index_label, 'centroid_z'] = mean_result[2]
    
    cur_label = df_db_labels[df_db_labels.label == dic_label]
    min_distance_id = -1
    min_distance_value = 1000
    for idx, row in df_db_result[df_db_result.label == dic_label].iterrows():
        if np.sum((np.array(cur_label[['centroid_x', 'centroid_y', 'centroid_z']]) - np.array(row[[0,1,2]])), axis=1) ** 2 < min_distance_value:
            min_distance_value = np.sum((np.array(cur_label[['centroid_x', 'centroid_y', 'centroid_z']]) - np.array(row[[0,1,2]])), axis=1) ** 2
            min_distance_id = idx
    if min_distance_id > -1:
        json_champions = json.loads(df_read.loc[min_distance_id, "champions"].replace("'", '"'))
        json_traits = json.loads(df_read.loc[min_distance_id, "traits"].replace("'", '"'))
        names = [staticdata_manager.GetChampionKoreanName(champion['character_id']) for champion in json_champions]
        df_db_labels.loc[index_label, 'champions'] = str(names)
        names.clear()
        for trait in json_traits:
            if trait['tier_current'] > 0:
                names.append(staticdata_manager.GetTraitKoreanName(trait['name']))
        df_db_labels.loc[index_label, 'traits'] = str(names)
        
    if len(df_read[df_read.label == dic_label].index) == 0:
        df_db_labels.loc[index_label, 'win_rate'] = 0.00
        df_db_labels.loc[index_label, 'defence_rate'] = 0.00
    else:
        df_db_labels.loc[index_label, 'win_rate'] = round(len(df_read[(df_read.label == dic['label']) & (df_read.placement == 1)].index) / len(df_read[df_read.label == dic['label']].index) * 100, 2)
        df_db_labels.loc[index_label, 'defence_rate'] = round(len(df_read[(df_read.label == dic['label']) & (df_read.placement <= 4)].index) / len(df_read[df_read.label == dic['label']].index) * 100, 2)
    
    if len(df_matchtime_gametype[(df_matchtime_gametype.label == dic_label)&
                                 (df_matchtime_gametype.gametype_id == 1)].index) == 0:
        df_db_labels.loc[index_label, 'turbo_win_rate'] = 0.00
        df_db_labels.loc[index_label, 'turbo_defence_rate'] = 0.00
    else:
        df_db_labels.loc[index_label, 'turbo_win_rate'] = round(len(df_matchtime_gametype[(df_matchtime_gametype.label == dic_label) &
                                                                                          (df_matchtime_gametype.placement == 1) &
                                                                                         (df_matchtime_gametype.gametype_id == 1)].index) /
                                                                len(df_matchtime_gametype[(df_matchtime_gametype.label == dic_label) &
                                                                                         (df_matchtime_gametype.gametype_id == 1)].index) * 100, 2)
        df_db_labels.loc[index_label, 'turbo_defence_rate'] = round(len(df_matchtime_gametype[(df_matchtime_gametype.label == dic['label']) & 
                                                                                              (df_matchtime_gametype.placement <= 4) &
                                                                                              (df_matchtime_gametype.gametype_id == 1)].index) / 
                                                                    len(df_matchtime_gametype[(df_matchtime_gametype.label == dic_label) &
                                                                                              (df_matchtime_gametype.gametype_id == 1)].index) * 100, 2)

    if len(df_matchtime_gametype[(df_matchtime_gametype.label == dic_label)&
                                 (df_matchtime_gametype.gametype_id == 2)].index) == 0:
        df_db_labels.loc[index_label, 'standard_win_rate'] = 0.00
        df_db_labels.loc[index_label, 'standard_defence_rate'] = 0.00
    else:
        df_db_labels.loc[index_label, 'standard_win_rate'] = round(len(df_matchtime_gametype[(df_matchtime_gametype.label == dic_label) &
                                                                                          (df_matchtime_gametype.placement == 1) &
                                                                                         (df_matchtime_gametype.gametype_id == 2)].index) /
                                                                len(df_matchtime_gametype[(df_matchtime_gametype.label == dic_label) &
                                                                                         (df_matchtime_gametype.gametype_id == 2)].index) * 100, 2)
        df_db_labels.loc[index_label, 'standard_defence_rate'] = round(len(df_matchtime_gametype[(df_matchtime_gametype.label == dic['label']) & 
                                                                                              (df_matchtime_gametype.placement <= 4) &
                                                                                              (df_matchtime_gametype.gametype_id == 2)].index) / 
                                                                    len(df_matchtime_gametype[(df_matchtime_gametype.label == dic_label) &
                                                                                              (df_matchtime_gametype.gametype_id == 2)].index) * 100, 2)

    period_win_rate = []
    period_defence_rate = []
    turbo_period_win_rate = []
    turbo_period_defence_rate = []
    standard_period_win_rate = []
    standard_period_defence_rate = []
    daily_win_rate = []
    daily_defence_rate = []
    turbo_daily_win_rate = []
    turbo_daily_defence_rate = []
    standard_daily_win_rate = []
    standard_daily_defence_rate = []
    for s_time, e_time in duration_datetime(start_time, end_time, period_timedelta):
        df_matches = df_matchtime_gametype[(df_matchtime_gametype.label == dic['label']) & 
                                      (df_matchtime_gametype.matched_at >= s_time) & 
                                      (df_matchtime_gametype.matched_at < e_time)]
        df_turbo_matches = df_matches[df_matches.gametype_id == 1]
        df_standard_matches = df_matches[df_matches.gametype_id == 2]
        
        if len(df_matches.index) == 0:
            period_win_rate.append(0.00)
            period_defence_rate.append(0.00)
        else:
            period_win_rate.append(round(len(df_matches[(df_matches.placement == 1)].index) / len(df_matches.index) * 100, 2))
            period_defence_rate.append(round(len(df_matches[(df_matches.placement <= 4)].index) / len(df_matches.index) * 100, 2))
        if len(df_turbo_matches.index) == 0:
            turbo_period_win_rate.append(0.00)
            turbo_period_defence_rate.append(0.00)
        else:
            turbo_period_win_rate.append(round(len(df_turbo_matches[(df_turbo_matches.placement == 1)].index) / len(df_turbo_matches.index) * 100, 2))
            turbo_period_defence_rate.append(round(len(df_turbo_matches[(df_turbo_matches.placement <= 4)].index) / len(df_turbo_matches.index) * 100, 2))
        if len(df_standard_matches.index) == 0:
            standard_period_win_rate.append(0.00)
            standard_period_defence_rate.append(0.00)
        else:
            standard_period_win_rate.append(round(len(df_standard_matches[(df_standard_matches.placement == 1)].index) / len(df_standard_matches.index) * 100, 2))
            standard_period_defence_rate.append(round(len(df_standard_matches[(df_standard_matches.placement <= 4)].index) / len(df_standard_matches.index) * 100, 2))
            
    for s_time, e_time in duration_datetime(start_time, end_time, daily_timedelta):
        df_matches = df_matchtime_gametype[(df_matchtime_gametype.label == dic['label']) & 
                                      (df_matchtime_gametype.matched_at >= s_time) & 
                                      (df_matchtime_gametype.matched_at < e_time)]
        df_turbo_matches = df_matches[df_matches.gametype_id == 1]
        df_standard_matches = df_matches[df_matches.gametype_id == 2]
        
        if len(df_matches.index) == 0:
            daily_win_rate.append(0.00)
            daily_defence_rate.append(0.00)
        else:
            daily_win_rate.append(round(len(df_matches[(df_matches.placement == 1)].index) /len(df_matches.index) * 100, 2))
            daily_defence_rate.append(round(len(df_matches[(df_matches.placement <= 4)].index) /len(df_matches.index) * 100, 2))
        if len(df_turbo_matches.index) == 0:
            turbo_daily_win_rate.append(0.00)
            turbo_daily_defence_rate.append(0.00)
        else:
            turbo_daily_win_rate.append(round(len(df_turbo_matches[(df_turbo_matches.placement == 1)].index) / len(df_turbo_matches.index) * 100, 2))
            turbo_daily_defence_rate.append(round(len(df_turbo_matches[(df_turbo_matches.placement <= 4)].index) / len(df_turbo_matches.index) * 100, 2))
        if len(df_standard_matches.index) == 0:
            standard_daily_win_rate.append(0.00)
            standard_daily_defence_rate.append(0.00)
        else:
            standard_daily_win_rate.append(round(len(df_standard_matches[(df_standard_matches.placement == 1)].index) / len(df_standard_matches.index) * 100, 2))
            standard_daily_defence_rate.append(round(len(df_standard_matches[(df_standard_matches.placement <= 4)].index) / len(df_standard_matches.index) * 100, 2))
            
    df_db_labels.loc[index_label, 'period_win_rate'] = str(period_win_rate)
    df_db_labels.loc[index_label, 'period_defence_rate'] = str(period_defence_rate)
    df_db_labels.loc[index_label, 'turbo_period_win_rate'] = str(turbo_period_win_rate)
    df_db_labels.loc[index_label, 'turbo_period_defence_rate'] = str(turbo_period_defence_rate)
    df_db_labels.loc[index_label, 'standard_period_win_rate'] = str(standard_period_win_rate)
    df_db_labels.loc[index_label, 'standard_period_defence_rate'] = str(standard_period_defence_rate)
    df_db_labels.loc[index_label, 'daily_win_rate'] = str(daily_win_rate)
    df_db_labels.loc[index_label, 'daily_defence_rate'] = str(daily_defence_rate)
    df_db_labels.loc[index_label, 'turbo_daily_win_rate'] = str(turbo_daily_win_rate)
    df_db_labels.loc[index_label, 'turbo_daily_defence_rate'] = str(turbo_daily_defence_rate)
    df_db_labels.loc[index_label, 'standard_daily_win_rate'] = str(standard_daily_win_rate)
    df_db_labels.loc[index_label, 'standard_daily_defence_rate'] = str(standard_daily_defence_rate)

result = {
    'info' : {
        'version': tft_clustering_version,
        'start_time': start_time,
        'end_time': end_time,
        'period': period_win_rate
    },
    'data' : df_db_labels.to_dict()
}

In [9]:
df_db_labels

,label,counts,centroid_x,centroid_y,centroid_z,champions,traits,win_rate,defence_rate,turbo_win_rate,...,turbo_period_win_rate,turbo_period_defence_rate,standard_period_win_rate,standard_period_defence_rate,daily_win_rate,daily_defence_rate,turbo_daily_win_rate,turbo_daily_defence_rate,standard_daily_win_rate,standard_daily_defence_rate
0,-1,5154,-2.102544,1.063600,-0.587191,"['클레드', '뽀삐', '케넨', '모르가나', '룰루', '타릭', '아이번',...","['극악무도', '악동', '기원자', '기사', '신비술사']",10.81,46.35,11.08,...,"[8.29, 10.77, 12.77, 13.76]","[50.86, 54.53, 57.25, 56.88]","[10.27, 11.37, 10.2, 10.74]","[42.79, 42.29, 43.2, 42.28]",[10.81],[46.35],[11.08],[54.77],[10.69],[42.69]
1,0,998,2.801470,7.641134,4.993144,"['리산드라', '르블랑', '녹턴', '모르가나', '아이번', '다이애나', '...","['암살자', '악의 여단', '영겁', '신비술사', '어둠의 인도자', '재생술...",16.23,59.12,15.52,...,"[15.38, 10.53, 16.67, 50.0]","[69.23, 47.37, 66.67, 100.0]","[16.17, 16.61, 15.94, 16.67]","[56.6, 58.98, 60.31, 60.0]",[16.23],[59.12],[15.52],[62.07],[16.28],[58.94]
2,1,7367,3.015653,-2.016354,-1.120162,"['헤카림', '카타리나', '드레이븐', '라이즈', '렐', '모데카이저', '...","['암살자', '기병대', '영겁', '망각', '군단', '신비술사']",13.48,53.21,18.49,...,"[21.9, 20.08, 15.83, 10.87]","[64.05, 63.98, 61.15, 60.87]","[12.1, 11.83, 13.0, 13.39]","[51.67, 51.54, 50.13, 51.97]",[13.48],[53.21],[18.49],[62.87],[12.41],[51.15]
3,2,1685,-5.070984,-3.382390,6.833163,"['칼리스타', '빅토르', '브랜드', '자이라', '럭스', '라이즈', '벨코...","['괴생명체', '신비술사', '구원받은 자', '주문술사']",11.63,57.98,18.84,...,"[23.81, 13.64, 9.52, 60.0]","[85.71, 63.64, 61.9, 60.0]","[13.35, 12.24, 9.62, 8.53]","[56.17, 58.26, 56.99, 60.47]",[11.63],[57.98],[18.84],[69.57],[11.32],[57.49]
4,3,416,-2.697538,2.658605,-3.753574,"['클레드', '뽀삐', '직스', '케넨', '룰루', '라이즈', '렐', '티모']","['기병대', '극악무도', '악동', '신비술사']",2.88,31.49,3.44,...,"[1.67, 2.02, 3.57, 15.79]","[30.0, 35.35, 30.95, 52.63]","[0.0, 3.64, 0.0, 12.5]","[15.0, 40.0, 25.49, 12.5]",[2.88],[31.49],[3.44],[33.97],[1.95],[27.27]
5,4,802,-4.714238,6.747065,-4.458917,"['클레드', '뽀삐', '케넨', '룰루', '럭스', '라이즈', '티모', '...","['용 사육사', '극악무도', '악동', '신비술사']",29.68,73.57,35.21,...,"[31.25, 30.43, 40.0, 44.44]","[84.38, 84.78, 87.27, 100.0]","[27.4, 29.69, 28.95, 24.56]","[73.29, 72.05, 68.86, 66.67]",[29.68],[73.57],[35.21],[86.62],[28.48],[70.76]
6,5,117,-1.144593,7.344697,-0.522662,"['카직스', '그라가스', '소라카', '리븐', '니달리', '카르마', '볼리...","['싸움꾼', '빛의 인도자']",9.40,48.72,10.61,...,"[7.69, 7.69, 16.67, 0.0]","[53.85, 61.54, 50.0, 33.33]","[27.27, 4.17, 0.0, 0.0]","[54.55, 50.0, 23.08, 0.0]",[9.4],[48.72],[10.61],[54.55],[7.84],[41.18]
7,6,32,-5.220605,5.839310,-2.523521,"['타릭', '카르마', '아이번', '라이즈', '렐', '볼리베어', '가렌',...","['용 사육사', '빛의 인도자', '신왕', '기원자', '기사', '재생술사',...",18.75,65.62,0.00,...,"[0.0, 0.0, 0.0, 0.0]","[100.0, 100.0, 0.0, 0.0]","[30.0, 9.09, 25.0, 0.0]","[70.0, 63.64, 62.5, 0.0]",[18.75],[65.62],[0.0],[100.0],[20.0],[63.33]
8,7,34,-3.327873,-2.532159,4.317243,"['클레드', '빅토르', '브랜드', '세주아니', '헤카림', '자이라', '벨...","['기병대', '주문술사']",17.65,47.06,11.11,...,"[0.0, 33.33, 0.0, 0.0]","[33.33, 100.0, 100.0, 0.0]","[33.33, 25.0, 12.5, 16.67]","[66.67, 25.0, 25.0, 50.0]",[17.65],[47.06],[11.11],[77.78],[20.0],[36.0]
9,8,31,-2.421613,7.634541,-1.477013,"['그라가스', '소라카', '리븐', '니달리', '카르마', '아이번', '볼리...","['싸움꾼', '빛의 인도자', '신왕', '기원자', '재생술사', '망령']",12.90,70.97,50.00,...,"[0.0, 50.0, 50.0, 0.0]","[0.0, 100.0, 100.0, 0.0]","[0.0, 9.09, 0.0, 0.0]","[60.0, 54.55, 85.71, 50.0]",[12.9],[70.97],[50.0],[100.0],[4.0],[64.0]


In [151]:
df_read

,id,Set5_Assassin_num_units,Set5_Assassin_tier_current,Set5_Brawler_num_units,Set5_Brawler_tier_current,Set5_Caretaker_num_units,Set5_Caretaker_tier_current,Set5_Coven_num_units,Set5_Coven_tier_current,Set5_Dawnbringer_num_units,...,level,placement,players_eliminated,time_eliminated,total_damage_to_players,traits,champions,is_analyzed,match_id,label
0,269633,4,2,1,0,0,0,3,1,0,...,8,3,0,1894,144,"[{'name': 'Set5_Assassin', 'num_units': 4, 'st...","[{'character_id': 'TFT5_Lissandra', 'items': [...",True,34611,0.0
1,269634,0,0,0,0,0,0,0,0,1,...,8,4,2,1804,108,"[{'name': 'Set5_Cavalier', 'num_units': 1, 'st...","[{'character_id': 'TFT5_Nautilus', 'items': [3...",True,34611,1.0
2,269635,2,1,0,0,0,0,1,0,0,...,9,2,2,2087,164,"[{'name': 'Set5_Abomination', 'num_units': 1, ...","[{'character_id': 'TFT5_Morgana', 'items': [],...",True,34611,1.0
3,269636,0,0,1,0,0,0,0,0,0,...,9,1,1,2087,199,"[{'name': 'Set5_Brawler', 'num_units': 1, 'sty...","[{'character_id': 'TFT5_Kled', 'items': [67], ...",True,34611,5.0
4,269637,2,1,0,0,0,0,0,0,0,...,7,8,0,1486,16,"[{'name': 'Set5_Abomination', 'num_units': 1, ...","[{'character_id': 'TFT5_Trundle', 'items': [],...",True,34611,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16716,343226,0,0,1,0,0,0,0,0,0,...,8,7,1,903,14,"[{'name': 'Set5_Brawler', 'num_units': 1, 'sty...","[{'character_id': 'TFT5_Syndra', 'items': [7],...",True,44247,NaN
16717,343227,1,0,0,0,0,0,0,0,0,...,9,3,1,1164,30,"[{'name': 'Set5_Assassin', 'num_units': 1, 'st...","[{'character_id': 'TFT5_Kled', 'items': [16, 5...",True,44247,NaN
16718,343228,0,0,0,0,0,0,0,0,1,...,9,2,1,1231,30,"[{'name': 'Set5_Cavalier', 'num_units': 1, 'st...","[{'character_id': 'TFT5_Leona', 'items': [], '...",True,44247,NaN
16719,343229,0,0,0,0,0,0,0,0,0,...,9,1,3,1236,34,"[{'name': 'Set5_Abomination', 'num_units': 1, ...","[{'character_id': 'TFT5_Kled', 'items': [29], ...",True,44247,NaN


In [10]:
df_matchtime_gametype

,placement,match_id,label,id,matched_at,gametype_id
0,3,34611,0,34611,2021-05-18 11:49:35,2
1,4,34611,1,34611,2021-05-18 11:49:35,2
2,2,34611,1,34611,2021-05-18 11:49:35,2
3,1,34611,4,34611,2021-05-18 11:49:35,2
4,8,34611,1,34611,2021-05-18 11:49:35,2
...,...,...,...,...,...,...
16716,7,44247,-1,44247,2021-05-18 10:35:06,1
16717,3,44247,4,44247,2021-05-18 10:35:06,1
16718,2,44247,-1,44247,2021-05-18 10:35:06,1
16719,1,44247,-1,44247,2021-05-18 10:35:06,1


In [92]:
df_db_labels

,label,counts,centroid_x,centroid_y,centroid_z,champions,traits,win_rate,defence_rate
0,-1,5069,-2.104191,1.070935,-0.690839,"['노틸러스', '모르가나', '잭스', '드레이븐', '라이즈', '렐', '비에...","['망각', '신왕', '철갑', '기사', '신비술사', '어둠의 인도자', '척...",10.73,46.18
1,0,974,2.758449,7.648668,4.960747,"['리산드라', '르블랑', '녹턴', '모르가나', '카타리나', '럭스', '아...","['암살자', '악의 여단', '신비술사', '어둠의 인도자', '재생술사', '망령']",16.43,59.03
2,1,6939,3.049483,-1.983578,-1.167274,"['노틸러스', '쓰레쉬', '카타리나', '드레이븐', '다이애나', '렐', '...","['암살자', '용 사냥꾼', '망각', '철갑', '기사', '군단']",13.45,53.16
3,2,1646,-5.118361,-3.532436,6.812154,"['빅토르', '브랜드', '자이라', '누누와 윌럼프', '라이즈', '벨코즈',...","['괴생명체', '영겁', '철갑', '신비술사', '주문술사']",11.66,58.14
4,3,298,-2.854829,2.958329,-3.684177,"['클레드', '뽀삐', '직스', '케넨', '룰루', '럭스', '렐', '하이...","['용 사육사', '기병대', '악동', '신비술사']",2.35,32.89
5,4,45,-1.958804,1.610065,-3.079520,"['클레드', '뽀삐', '직스', '세주아니', '케넨', '헤카림', '룰루',...","['기병대', '악동']",6.67,31.11
6,5,740,-4.766430,6.826784,-4.267853,"['클레드', '뽀삐', '케넨', '룰루', '아이번', '벨코즈', '티모', ...","['극악무도', '빛의 인도자', '신왕', '악동', '기원자', '기사', '망령']",29.86,73.51
7,6,96,-1.189659,7.370125,-0.529142,"['카직스', '그라가스', '소라카', '리븐', '니달리', '카르마', '아이...","['암살자', '빛의 인도자', '신왕', '기원자', '재생술사', '척후병']",8.33,45.83
8,7,32,-5.271377,5.888531,-2.580695,"['타릭', '카르마', '아이번', '라이즈', '렐', '볼리베어', '가렌',...","['용 사육사', '빛의 인도자', '신왕', '기원자', '기사', '재생술사',...",18.75,65.62
9,8,31,6.386198,-0.412208,-0.011243,"['세주아니', '트런들', '야스오', '모르가나', '리 신', '판테온', '...","['용 사냥꾼', '군단', '어둠의 인도자', '척후병']",9.68,32.26


In [40]:
df_db_result

,0,1,2,label,id
0,2.270340,10.038015,4.976179,0,269633
1,-4.919173,-4.144554,-4.994747,1,269634
2,0.742612,-0.664258,0.169259,1,269635
3,-4.322770,7.356429,-3.020756,5,269636
4,5.684949,-1.235000,1.242026,1,269637
...,...,...,...,...,...
15928,5.156965,-1.693068,-2.458487,1,323850
15929,-5.491712,-4.546916,-4.287839,1,323851
15930,-0.488269,-2.487304,2.633696,1,323852
15931,-1.683782,5.604318,-2.622104,-1,323853


# 결과 가시화

In [168]:
from lolchess.StaticDataManager import StaticDataManager
sdb = StaticDataManager()

In [160]:
df_result = pd.concat([df, pd.Series(db_labels, name='db_labels')], axis=1)
df_result = pd.concat([df_result, pd.Series(kmeans_labels, name='kmeans_labels')], axis=1)
df_deck_result = df_result.drop(['id',
                           'match_id',
                           'gold_left',
                           'last_round', 
                           'level',
                           'players_eliminated', 
                           'time_eliminated', 
                           'total_damage_to_players', 
                           'traits', 
                           'champions'], axis=1)
#df_deck_result.loc[df_deck_result['db_labels'] == 0].groupby('placement', as_index=False).size().loc[0,'size']

In [166]:
df_deck_result.groupby('db_labels', as_index=False)

In [163]:
df_traits_result = df_deck_result.groupby(['db_labels','placement'],as_index=False).mean()
df_traits_result['placement']

0      1
1      2
2      3
3      4
4      5
      ..
114    4
115    5
116    6
117    7
118    8
Name: placement, Length: 119, dtype: int64

In [165]:
df_traits_result.head(10)

,db_labels,placement,Set5_Assassin_num_units,Set5_Assassin_tier_current,Set5_Brawler_num_units,Set5_Brawler_tier_current,Set5_Caretaker_num_units,Set5_Caretaker_tier_current,Set5_Coven_num_units,Set5_Coven_tier_current,...,TFT5_Syndra_exits,TFT5_Karma_tier,TFT5_Karma_exits,TFT5_Sett_tier,TFT5_Sett_exits,TFT5_Gragas_tier,TFT5_Gragas_exits,TFT5_Riven_tier,TFT5_Riven_exits,kmeans_labels
0,-1,1,0.583199,0.205170,0.668821,0.171244,0.368336,0.368336,0.258481,0.038772,...,0.114701,0.305331,0.159935,0.250404,0.090468,0.126010,0.059774,0.187399,0.088853,6.794830
1,-1,2,0.658284,0.227811,0.642012,0.183432,0.323964,0.323964,0.313609,0.050296,...,0.105030,0.213018,0.115385,0.186391,0.073964,0.124260,0.057692,0.142012,0.073964,7.221893
2,-1,3,0.562678,0.183761,0.745014,0.195157,0.294872,0.294872,0.257835,0.035613,...,0.121083,0.311966,0.170940,0.259259,0.105413,0.143875,0.069801,0.189459,0.103989,7.125356
3,-1,4,0.575209,0.186630,0.660167,0.183844,0.299443,0.299443,0.352368,0.068245,...,0.101671,0.253482,0.143454,0.199164,0.083565,0.165738,0.082173,0.161560,0.083565,7.069638
4,-1,5,0.582633,0.184874,0.665266,0.200280,0.238095,0.238095,0.343137,0.060224,...,0.120448,0.337535,0.193277,0.179272,0.079832,0.184874,0.098039,0.154062,0.082633,7.077031
5,-1,6,0.590226,0.180451,0.685464,0.210526,0.229323,0.229323,0.333333,0.050125,...,0.125313,0.298246,0.177945,0.240602,0.109023,0.216792,0.107769,0.204261,0.114035,7.172932
6,-1,7,0.539340,0.168782,0.692893,0.205584,0.246193,0.246193,0.318528,0.043147,...,0.093909,0.302030,0.184010,0.269036,0.125635,0.243655,0.126904,0.157360,0.096447,7.354061
7,-1,8,0.463255,0.140420,0.624672,0.182415,0.221785,0.221785,0.284777,0.036745,...,0.112861,0.244094,0.154856,0.288714,0.141732,0.171916,0.099738,0.146982,0.089239,7.010499
8,0,1,0.884141,0.330484,0.052232,0.000950,0.024691,0.024691,0.217474,0.000950,...,0.002849,0.004748,0.003799,0.000000,0.000000,0.000950,0.000950,0.042735,0.021842,5.338082
9,0,2,1.142276,0.410569,0.050813,0.000000,0.021341,0.021341,0.218496,0.002033,...,0.003049,0.001016,0.001016,0.000000,0.000000,0.000000,0.000000,0.053862,0.027439,5.273374


In [156]:
print(df_traits_result.loc[2].sort_values(ascending=False).head(30))

match_id                         967.199584
placement                          4.301455
Set5_Spellweaver_num_units         3.880457
Set5_Abomination_num_units         2.913721
Set5_Redeemed_num_units            2.743243
Set5_Mystic_num_units              2.094595
Set5_Spellweaver_tier_current      1.920998
TFT5_Velkoz_tier                   1.847193
TFT5_Brand_tier                    1.822245
Set5_Forgotten_num_units           1.794179
TFT5_Nunu_tier                     1.696466
TFT5_Zyra_tier                     1.651767
TFT5_Ryze_tier                     1.512474
TFT5_Viktor_tier                   1.476091
TFT5_Lux_tier                      1.458420
TFT5_Rell_tier                     1.380457
Set5_Mystic_tier_current           1.103950
db_labels                          1.000000
Set5_Brawler_num_units             0.993763
TFT5_Velkoz_exits                  0.991684
TFT5_Brand_exits                   0.983368
Set5_Abomination_tier_current      0.971933
Set5_Ironclad_num_units         

In [169]:
df_final_result = df_deck_result.groupby('label',as_index=False).size()

KeyError: 'label'

In [283]:
df_final_result.loc[1, 'traits'] = '(빛의 인도자, 군단), (카르마, 가렌, 아이번, 볼리베어)'
df_final_result.loc[2, 'traits'] = '(어둠의 인도자, 군단), (아펠리오스, 다이애나, 다리우스, 모르가나)'
df_final_result.loc[3, 'traits'] = '(구원받은 자, 군단), (렐, 벨코즈, 케일, 레오나)'
df_final_result.loc[4, 'traits'] = '(망각, 군단), (드레이븐, 라이즈, 비에고, 쓰레쉬)'
df_final_result.loc[5, 'traits'] = '(용족, 용 사육사), (하이머딩거, 킨드레드, 볼리베어, 애쉬)'
df_final_result.loc[6, 'traits'] = '(암살자, 어둠의 인도자), (다이애나, 르블랑, 아이번, 비에고)'
df_final_result.loc[7, 'traits'] = '(척후병, 어둠의 인도자 혹은 빛의 인도자), (잭스, 비에고, 판테온, 트런들)'
df_final_result.loc[8, 'traits'] = '(재생술사, 어둠의 인도자 혹은 빛의 인도자), (아이번, 소라카, 리산드라, 블라디미르)'
df_final_result.loc[9, 'traits'] = '(악동, 기병대), (뽀삐, 직스, 클레드, 케넨)'
df_final_result.loc[10, 'traits'] = '(어둠의 인도자, 정찰대), (킨드레드, 다리우스, 타릭, 아펠리오스)'
df_final_result.loc[11, 'traits'] = '(용족, 괴생명체), (하이머딩거, 라이즈, 자이라, 세트)'
#df_final_result.loc[12, 'traits'] = '(Redeemed, Legionnaire), (Rell, Velkoz, Kayle, Leona)'
df_final_result.loc[13, 'traits'] = '(용족, 괴생명체), (라이즈, 누누, 자이라, 세트)'
df_final_result.loc[14, 'traits'] = '(암살자, 어둠의 인도자 혹은 빛의 인도자), (비에고, 다이애나, 아이번, 볼리베어)'
df_final_result.loc[15, 'traits'] = '(어둠의 인도자, 용 사냥꾼), (다이애나, 모데카이저, 르블랑, 리산드라)'
df_final_result.loc[16, 'traits'] = '(악동, 기병대), (티모, 렐, 잭스, 뽀삐)'
df_final_result.loc[17, 'traits'] = '(어둠의 인도자, 정찰대), (다리우스, 킨드레드, 아펠리오스, 타릭)'
df_final_result.loc[18, 'traits'] = '(군단, 빛의 인도자), (모데카이저, 드레이븐, 리븐, 야스오)'
df_final_result.loc[19, 'traits'] = '(어둠의 인도자, 악의 여단), (다이애나, 리산드라, 모르가나, 블라디미르)'
df_final_result.loc[20, 'traits'] = '(군단, 어둠의 인도자), (드레이븐, 라이즈, 다이애나, 렐)'
df_final_result['평균순위'] = round(df_traits_result['placement'], 2)
df_final_result['1등횟수'] = x
df_final_result['승률'] = round((df_final_result['1등횟수'] / df_final_result['size']) * 100, 2)
df_final_result

,label,size,traits,평균순위,1등횟수,승률
0,-1,10666,NaN,4.49,0,0.00
1,0,1045,"(빛의 인도자, 군단), (카르마, 가렌, 아이번, 볼리베어)",4.20,157,15.02
2,1,914,"(어둠의 인도자, 군단), (아펠리오스, 다이애나, 다리우스, 모르가나)",4.30,109,11.93
3,2,601,"(구원받은 자, 군단), (렐, 벨코즈, 케일, 레오나)",4.80,54,8.99
4,3,1477,"(망각, 군단), (드레이븐, 라이즈, 비에고, 쓰레쉬)",4.66,130,8.80
5,4,131,"(용족, 용 사육사), (하이머딩거, 킨드레드, 볼리베어, 애쉬)",4.32,24,18.32
6,5,308,"(암살자, 어둠의 인도자), (다이애나, 르블랑, 아이번, 비에고)",3.82,70,22.73
7,6,414,"(척후병, 어둠의 인도자 혹은 빛의 인도자), (잭스, 비에고, 판테온, 트런들)",4.26,38,9.18
8,7,35,"(재생술사, 어둠의 인도자 혹은 빛의 인도자), (아이번, 소라카, 리산드라, 블라...",5.97,1,2.86
9,8,228,"(악동, 기병대), (뽀삐, 직스, 클레드, 케넨)",5.33,12,5.26


In [286]:
df_final_result.columns = ['라벨', '양', '시너지 및 챔피언', '평균순위', '1등횟수', '승률']
df_final_result = df_final_result.drop([0,12])
df_final_result = df_final_result.drop('라벨', axis=1)

In [294]:
df_final_result.sort_values(by='양', ascending=False).reset_index().drop('index', axis=1)

,양,시너지 및 챔피언,평균순위,1등횟수,승률
0,1477,"(망각, 군단), (드레이븐, 라이즈, 비에고, 쓰레쉬)",4.66,130,8.80
1,1045,"(빛의 인도자, 군단), (카르마, 가렌, 아이번, 볼리베어)",4.20,157,15.02
2,914,"(어둠의 인도자, 군단), (아펠리오스, 다이애나, 다리우스, 모르가나)",4.30,109,11.93
3,601,"(구원받은 자, 군단), (렐, 벨코즈, 케일, 레오나)",4.80,54,8.99
4,414,"(척후병, 어둠의 인도자 혹은 빛의 인도자), (잭스, 비에고, 판테온, 트런들)",4.26,38,9.18
5,308,"(암살자, 어둠의 인도자), (다이애나, 르블랑, 아이번, 비에고)",3.82,70,22.73
6,228,"(악동, 기병대), (뽀삐, 직스, 클레드, 케넨)",5.33,12,5.26
7,194,"(악동, 기병대), (티모, 렐, 잭스, 뽀삐)",3.31,53,27.32
8,187,"(암살자, 어둠의 인도자 혹은 빛의 인도자), (비에고, 다이애나, 아이번, 볼리베어)",4.21,30,16.04
9,178,"(용족, 괴생명체), (하이머딩거, 라이즈, 자이라, 세트)",5.21,8,4.49


## 아이템 분석

In [94]:
from sklearn.preprocessing import MultiLabelBinarizer

In [102]:
def GetItems(match):
    if match['metadata']['data_version'] != '5':
        return pd.DataFrame()
    df_participants = pd.DataFrame(match['info']['participants'])
    df_participants['list_items'] = np.empty((len(df_participants), 0)).tolist()
    for idx_participant, df_participant in df_participants.iterrows():
        for idx, row in pd.DataFrame(df_participant['traits']).iterrows():
            if not 'Set5' in row['name']:
                return pd.DataFrame()
    
        df_participant_units = pd.DataFrame(df_participant['units'])
        for idx, row in df_participant_units.iterrows():
            df_participants.iloc[idx_participant]['list_items'].extend(row['items'])
            
    return df_participants

def GetItemOntHot(matches):
    if matches == None:
        return None
    df_items = GetItems(matches[0])
    for match in matches[1:]:
        df_temp_items = GetItems(match)
        if df_temp_items.empty:
            continue
        df_items = pd.concat([df_items, df_temp_items], ignore_index=True, sort=False).fillna(0)
    
    mlb = MultiLabelBinarizer(sparse_output=True)
    df_items = df_items.join(
                    pd.DataFrame.sparse.from_spmatrix(
                    mlb.fit_transform(df_items.pop('list_items')),
                    index=df_items.index,
                    columns=mlb.classes_))
    return df_items

In [122]:
df_items = GetItemOntHot(matches).drop(['companion', 
                                       'gold_left',
                                       'last_round', 
                                       'level',
                                       'players_eliminated', 
                                       'puuid', 
                                       'time_eliminated', 
                                       'total_damage_to_players', 
                                       'traits', 
                                       'units'], axis=1)

In [147]:
df_items.groupby(['placement'],as_index=False).sum()

,placement,1,2,3,4,5,6,7,9,11,...,1059,1066,1067,1069,1077,1099,1148,1168,1178,1189
0,1,0,0,0,0,1,0,1,1,1,...,1,0,1,1,0,0,0,1,0,1
1,2,1,0,1,1,1,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0
2,3,1,1,0,1,0,0,2,0,1,...,0,1,0,1,1,0,0,0,0,0
3,4,0,1,0,2,0,2,2,0,0,...,1,1,0,0,0,0,1,0,1,0
4,5,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,1,0,0,0,0,0,2,0,1,...,0,0,1,0,0,1,0,0,0,0
6,7,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
7,8,0,0,1,0,0,0,1,1,0,...,0,0,0,1,1,0,0,1,0,0


# 버려진 코드

In [ ]:
def GetPostParticipant(match):
    if match['metadata']['data_version'] != '5':
        return pd.DataFrame()
    df_participants = pd.DataFrame(match['info']['participants'])
    for idx_participant, df_participant in df_participants.iterrows():
        # 시너지(traits)
        df_participant_traits = pd.DataFrame(df_participant['traits'])
        df_participant_traits_splited = pd.DataFrame([0])
        for idx, row in df_participant_traits.iterrows():
            df_participant_traits_trait = df_participant_traits.loc[idx:idx].drop(['name', 'tier_total'], axis=1).add_prefix(row['name'] + '_')
            df_participant_traits_trait.index = [0]
            df_participant_traits_splited = df_participant_traits_splited.join(df_participant_traits_trait)
        df_participant_traits_splited.index = [idx_participant]
        df_participant_traits_splited = df_participant_traits_splited.drop(0, axis=1)
        df_participants = pd.concat([df_participants, df_participant_traits_splited], axis=1).fillna(0)
        
        # 유닛들(units)
        df_participant_units = pd.DataFrame(df_participant['units'])
        df_participant_units_splited = pd.DataFrame([0])
        for idx, row in df_participant_units.iterrows():
            df_participant_units_unit = df_participant_units.loc[idx:idx].drop(['character_id', 'items', 'name', 'rarity'], axis=1)
            if 'chosen' in df_participant_units.loc[idx:idx].columns.tolist():
                df_participant_units_unit = df_participant_units_unit.drop(['chosen'], axis=1)
            df_participant_units_unit = df_participant_units_unit.add_prefix(row['character_id'] + '_')
            df_participant_units_unit.index = [0]
            df_participant_units_splited = pd.concat([df_participant_units_splited, df_participant_units_unit], axis=1).fillna(0)
        df_participant_units_splited = df_participant_units_splited.T.groupby(level=0).agg(max).T
        df_participant_units_splited.index = [idx_participant]
        df_participant_units_splited = df_participant_units_splited.drop(0, axis=1)
        df_participants = pd.concat([df_participants, df_participant_units_splited], axis=1).fillna(0)
        
    df_participants = df_participants.T.groupby(level=0).agg(max).T
    return df_participants

In [ ]:
df_match = GetPostParticipant(matches[0])
for match in matches[1:]:
    df_temp_match = GetPostParticipant(match)
    if df_temp_match.empty:
        continue
    df_match = pd.concat([df_match, df_temp_match], ignore_index=True, sort=False)
    df_match = df_match.fillna(0)

In [ ]:
pca_kmean = PCA(n_components=3)
X_kmean_fit_pca = pca_kmean.fit_transform(df_clustering)

kmeans_PCA = KMeans(n_clusters=db_n_clusters_, init='k-means++', max_iter=300, n_init=db_n_clusters_, random_state= 3)

kmeans_labels = kmeans_PCA.fit_predict(X_kmean_fit_pca)
unique, counts = np.unique(kmeans_labels, return_counts=True)
for u, c in zip(unique, counts):
    print('label ' + str(u) + ": " + str(c))
count_kmeans_labels = dict(zip(unique, counts))

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_kmean_fit_pca[:,0],X_kmean_fit_pca[:,1],X_kmean_fit_pca[:,2], 
            c=kmeans_PCA, cmap='viridis',
            edgecolor='k', s=40, alpha = 0.5)


ax.set_title("KMean Analyze")
ax.dist = 10

ax.scatter(kmeans_PCA.cluster_centers_[:,0], kmeans_PCA.cluster_centers_[:,1], 
           kmeans_PCA.cluster_centers_[:,2], 
           s = 300, c = 'r', marker='*', label = 'Centroid')

plt.autoscale(enable=True, axis='x', tight=True)    

plt.show()